# 1 Prepare

In [1]:
import pandas
import numpy
import os
import networkx as nx

def GenFileName(year, month):
    return os.path.join('Data', 'TradeData', 'exportX{}M{}.csv'.format(year, month))

def ToDG(matrix, areas_list):
    DG = nx.DiGraph()
    edges = list()
    for i in range(0, numpy.size(matrix, 0)):
        for j in range(0, numpy.size(matrix, 1)):
            if ((matrix[i, j] != 0) and (ISO3[i] in areas_list) and (ISO3[j] in areas_list)):
                    edges.append((ISO3[i], ISO3[j], matrix[i, j]))
    DG.add_weighted_edges_from(edges)
    return DG

TradeKeys = pandas.read_excel('Data/Keys.xlsx', sheet_name = 'TradeNameToISO3')
RegionsTable = pandas.read_excel('Data/Keys.xlsx', sheet_name = 'Regions')

ISO3 = TradeKeys['ISO3']

AreasInRegions = {
    'World': ISO3.tolist()
}
for region in RegionsTable['REGION_CODE'].unique():
    AreasInRegions.update({
        region: RegionsTable.query('REGION_CODE == @region')['ISO3'].tolist()
    })
    
TradeMatrixs = {}
for year in range(2018, 2022):
    for month in range(1, 13):
        filename = GenFileName(year, month)
        mE = pandas.read_csv(filename, index_col = 0, encoding = 'ISO-8859-1').values
        mI = mE.T
        TradeMatrixs.update({
            (year, month): (mE, mI)
        })



# 2 Calculate

In [3]:
NtwDens = []
NodeAttrs = []

RegionPairsDone = list()

for region_x in AreasInRegions.keys():
    for region_y in AreasInRegions.keys():
        print(region_x + ' X ' + region_y, end = ': ')
        if (region_x == 'World' or region_y == 'World') and not (region_x == 'World' and region_y == 'World'):
            print('PASS')
            continue
        if ((region_y, region_x) in RegionPairsDone):
            print('PASS')
            continue
            
        areas = AreasInRegions[region_x] + AreasInRegions[region_y]
        for year in range(2018, 2022):
            for month in range(1, 13):
                mE, mI = TradeMatrixs[(year, month)]
                DGE = ToDG(mE, areas)
                DGI = ToDG(mI, areas)
                
                NtwDens.append({
                    'region_x': region_x,
                    'region_y': region_y,
                    'year': year,
                    'month': month,
                    'density': nx.density(DGE)
                })
                
                NodeAttrs.append({
                    'region_x': region_x,
                    'region_y': region_y,
                    'year': year,
                    'month': month,
                    'ex_vol': DGE.out_degree(weight = 'weight'),
                    'im_vol': DGI.out_degree(weight = 'weight'),
                    'ex_pr': nx.pagerank(DGE.reverse(), weight = 'weight'),
                    'im_pr': nx.pagerank(DGI.reverse(), weight = 'weight')
                })
                print((year, month), end = ' | ')
        print()
        
        RegionPairsDone.append((region_x, region_y))

World X World: (2018, 1) | (2018, 2) | (2018, 3) | (2018, 4) | (2018, 5) | (2018, 6) | (2018, 7) | (2018, 8) | (2018, 9) | (2018, 10) | (2018, 11) | (2018, 12) | (2019, 1) | (2019, 2) | (2019, 3) | (2019, 4) | (2019, 5) | (2019, 6) | (2019, 7) | (2019, 8) | (2019, 9) | (2019, 10) | (2019, 11) | (2019, 12) | (2020, 1) | (2020, 2) | (2020, 3) | (2020, 4) | (2020, 5) | (2020, 6) | (2020, 7) | (2020, 8) | (2020, 9) | (2020, 10) | (2020, 11) | (2020, 12) | (2021, 1) | (2021, 2) | (2021, 3) | (2021, 4) | (2021, 5) | (2021, 6) | (2021, 7) | (2021, 8) | (2021, 9) | (2021, 10) | (2021, 11) | (2021, 12) | 
World X APQ: PASS
World X DA: PASS
World X SSA: PASS
World X ADVEC: PASS
World X EU: PASS
World X WE: PASS
World X MECA: PASS
World X EAQ: PASS
World X EDE: PASS
World X SEQ: PASS
APQ X World: PASS
APQ X APQ: (2018, 1) | (2018, 2) | (2018, 3) | (2018, 4) | (2018, 5) | (2018, 6) | (2018, 7) | (2018, 8) | (2018, 9) | (2018, 10) | (2018, 11) | (2018, 12) | (2019, 1) | (2019, 2) | (2019, 3) | (201

# 3 Output

In [4]:
NtwDensDf = pandas.DataFrame(NtwDens)
NtwDensDf.to_excel('Tables/NtwDensTable.xlsx', index = False)

In [5]:
NodeAttrRecords = []
for nodeAttr in NodeAttrs:
    areas = list(set(AreasInRegions[nodeAttr['region_x']] + AreasInRegions[nodeAttr['region_y']]))
    for area in areas:
        NodeAttrRecords.append({
            'ISO3': area,
            'region_x': nodeAttr['region_x'],
            'region_y': nodeAttr['region_y'],
            'year': nodeAttr['year'],
            'month': nodeAttr['month'],
            'ex_vol': nodeAttr['ex_vol'][area],
            'im_vol': nodeAttr['im_vol'][area],
            'ex_pr': nodeAttr['ex_pr'][area],
            'im_pr': nodeAttr['im_pr'][area]
        })
NodeAttrDf = pandas.DataFrame(NodeAttrRecords)
NodeAttrDf.to_excel('Tables/NodeAttrTable.xlsx', index = False)